<a href="https://colab.research.google.com/github/awosoga/ssc2023/blob/main/Unfinished/DA_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
import os
from torch import optim
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from itertools import product
from tensorflow import summary

In [ ]:
input_path = "full_data.csv"  # Replace with your CSV file path


def read_data(input_path, debug=False, diff=False):
    """
    This function reads the timeseries data from a file and then normalizes and stationaize the signals. 

    Args:
        input_path (str): directory to dataset.

    Returns:
        X (np.ndarray): features.
        y (np.ndarray): ground truth.

    """
    df = pd.read_csv(input_path, nrows=251 if debug else None).groupby('provincename')

    # X = df.iloc[:, 0:-1].values
    province_data = []
    for name, group in df:
      X = group[["mean_max_temp_anomaly" , 
              'mean_min_temp_anomaly', 
              'mean_temp_anomaly', 
              'extr_max_temp_anomaly', 
              'extr_min_temp_anomaly', 
              'total_precip_anomaly',
              'total_rain_anomaly', 
              'total_snow_anomaly', 
              'snow_grnd_last_day_anomaly', 
              'co2_anomaly']].to_numpy()

      #X = df.drop(['Date', 'provincename', 'growth_rate', 'GeoUID'], axis=1).to_numpy()
      # y = df.iloc[:, -1].values
      y = np.array(group['growth_rate'])

      # print(X.shape)

      # X[:,4] = np.where(X[:,4]<-10, 0, X[:,4])
      ######################################################################################
      # Stationarize each signal
      # if diff:
      #     y = y[1:]-y[0:-1]
      #     X = X[1:,:] - X[0:-1,:]

      ##########################################################################################
      # Normalize every data
      # y = (y - min(y))
      # y /= max(y)

      Z = np.array(group['GeoUID']).reshape(-1,1)
      X = (X - np.min(X, axis=0))
      X /= np.max(X, axis=0)
      X = np.concatenate((Z, X), axis=1)
      
      province_data.append((X,y))

    return province_data

In [ ]:
class CustomLoss1(nn.Module):
    """
    A class which defines a custom loss function
    """
    def __init__(self):
        super().__init__()
        self.l1loss = nn.L1Loss()

    def forward(self, output, labels):
        """
        The loss function implemented is e^(3*abs(y_pred - y_true))
        """
        x = self.l1loss(output, labels)
        loss = torch.mean(torch.expm1(torch.mul(x, 3)))
        # print("X: ",x, " Loss: ", loss)
        return loss


class CustomLoss2(nn.Module):
    """
    A class which defines a custom loss function
    """
    def __init__(self):
        super().__init__()
        self.l1loss = nn.L1Loss()

    def forward(self, output, labels):
        """
        The loss function implemented is e^(1.5*(y_pred - y_true)^2)
        """
        x = self.l1loss(output, labels)
        loss = torch.mean(torch.expm1(torch.mul(x ** 2, 1.5)))
        # print("X: ",x, " Loss: ", loss)
        return loss


class CustomLoss3(nn.Module):
    def __init__(self):
        super().__init__()
        self.l2loss = nn.MSELoss()

    def forward(self, output, labels):
        """
        The loss function implemented is 3*(y_pred - y_true)^2
        """
        x = self.l2loss(output, labels)
        loss = torch.mul(x, 3)
        # print("X: ",x, " Loss: ", loss)
        return loss

In [ ]:
class Encoder(nn.Module):
    """Encoder in DA_RNN."""


    def __init__(self, T,
                 input_size,
                 encoder_num_hidden,
                 parallel=False):
        """
        Initialize an encoder in DA_RNN.
        
        Parameters
        ----------
        T: `int`
            The number of timesteps to consider attention upon.
        input_size:: `int`
            The dimension of the input
        encoder_num_hidden: `int`
            The dimension of the encoder's hidden state
        parallel: `bool`
            If set to `True` the training will be done parallely.
        
        Returns
        ------
        None
        """

        super(Encoder, self).__init__()
        self.encoder_num_hidden = encoder_num_hidden
        self.input_size = input_size
        self.parallel = parallel
        self.T = T

        # Fig 1. Temporal Attention Mechanism: Encoder is LSTM
        self.encoder_lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.encoder_num_hidden,
            num_layers = 1
        )

        # Construct Input Attention Mechanism via deterministic attention model
        # Eq. 8: W_e[h_{t-1}; s_{t-1}] + U_e * x^k
        self.encoder_attn = nn.Linear(
            in_features=2 * self.encoder_num_hidden + self.T - 1,
            out_features=1
        )


    def forward(self, X):
        """
        The forward propagation for the encoder.

        Parameters
        ----------
        X: `numpy.ndarray`
            input data
        
        Returns
        -------
        X_tilde: `numpy.ndarray`
            The input sequence of after forward pass thus after the application of attention
        X_encoded: `numpy.ndarray`
            The encoded sequence for the given input
        """

        X_tilde = Variable(X.data.new(
            X.size(0), self.T - 1, self.input_size).zero_())
        X_encoded = Variable(X.data.new(
            X.size(0), self.T - 1, self.encoder_num_hidden).zero_())

        # Eq. 8, parameters not in nn.Linear but to be learnt
        # v_e = torch.nn.Parameter(data=torch.empty(
        #     self.input_size, self.T).uniform_(0, 1), requires_grad=True)
        # U_e = torch.nn.Parameter(data=torch.empty(
        #     self.T, self.T).uniform_(0, 1), requires_grad=True)

        # h_n, s_n: initial states with dimention hidden_size
        h_n = self._init_states(X)
        s_n = self._init_states(X)

        for t in range(self.T - 1):
            # batch_size * input_size * (2 * hidden_size + T - 1)
            x = torch.cat((h_n.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           s_n.repeat(self.input_size, 1, 1).permute(1, 0, 2),
                           X.permute(0, 2, 1)), dim=2)

            x = self.encoder_attn(
                x.view(-1, self.encoder_num_hidden * 2 + self.T - 1))

            # get weights by softmax
            alpha = F.softmax(x.view(-1, self.input_size))

            # get new input for LSTM
            x_tilde = torch.mul(alpha, X[:, t, :])

            # Fix the warning about non-contiguous memory
            # https://discuss.pytorch.org/t/dataparallel-issue-with-flatten-parameter/8282
            self.encoder_lstm.flatten_parameters()

            # encoder LSTM
            _, final_state = self.encoder_lstm(x_tilde.unsqueeze(0), (h_n, s_n))
            h_n = final_state[0]
            s_n = final_state[1]

            X_tilde[:, t, :] = x_tilde
            X_encoded[:, t, :] = h_n

        return X_tilde, X_encoded


    def _init_states(self, X):
        """
        Initialize all 0 hidden states and cell states for encoder.

        Parameters
        ----------
        X: `numpy.ndarray`
            The input array

        Returns
        -------
            initial_hidden_states
        """

        # https://pytorch.org/docs/master/nn.html?#lstm
        return Variable(X.data.new(1, X.size(0), self.encoder_num_hidden).zero_())

In [ ]:
class Decoder(nn.Module):
    """Decoder in DA_RNN."""


    def __init__(self, T, decoder_num_hidden, encoder_num_hidden):
        """
        Initialize an decoder in DA_RNN.
        
        Parameters
        ----------
        T: `int`
            The number of timesteps to consider attention upon.
        decoder_num_hidden:: `int`
            The dimension of the decoder's hidden state
        encoder_num_hidden: `int`
            The dimension of the encoder's hidden state
        
        Returns
        ------
        None
        """

        super(Decoder, self).__init__()
        self.decoder_num_hidden = decoder_num_hidden
        self.encoder_num_hidden = encoder_num_hidden
        self.T = T

        self.attn_layer = nn.Sequential(
            nn.Linear(2 * decoder_num_hidden + encoder_num_hidden, encoder_num_hidden),
            nn.Tanh(),
            nn.Linear(encoder_num_hidden, 1)
        )
        self.lstm_layer = nn.LSTM(
            input_size=1,
            hidden_size=decoder_num_hidden
        )
        self.fc = nn.Linear(encoder_num_hidden + 1, 1)
        self.fc_final = nn.Linear(decoder_num_hidden + encoder_num_hidden, 1)

        self.fc.weight.data.normal_()


    def forward(self, X_encoded, y_prev):
        """
        The forward propagation for the decoder.

        Parameters
        ----------
        X_encoded: `numpy.ndarray`
            The input data after encoding.
        y_prev: `numpy.ndarray`
            The output in the previous timestep
        
        Returns
        -------
        y_pred: `numpy.ndarray`
            The predicted output for the current timestep
        """

        d_n = self._init_states(X_encoded)
        c_n = self._init_states(X_encoded)

        for t in range(self.T - 1):

            x = torch.cat((d_n.repeat(self.T - 1, 1, 1).permute(1, 0, 2),
                           c_n.repeat(self.T - 1, 1, 1).permute(1, 0, 2),
                           X_encoded), dim=2)

            beta = F.softmax(self.attn_layer(
                x.view(-1, 2 * self.decoder_num_hidden + self.encoder_num_hidden)).view(-1, self.T - 1))

            # Eqn. 14: compute context vector
            # batch_size * encoder_hidden_size
            context = torch.bmm(beta.unsqueeze(1), X_encoded)[:, 0, :]
            if t < self.T - 1:
                # Eqn. 15
                # batch_size * 1
                y_tilde = self.fc(
                    torch.cat((context, y_prev[:, t].unsqueeze(1)), dim=1))

                # Eqn. 16: LSTM
                self.lstm_layer.flatten_parameters()
                _, final_states = self.lstm_layer(
                    y_tilde.unsqueeze(0), (d_n, c_n))

                d_n = final_states[0]  # 1 * batch_size * decoder_num_hidden
                c_n = final_states[1]  # 1 * batch_size * decoder_num_hidden

        # Eqn. 22: final output
        y_pred = self.fc_final(torch.cat((d_n[0], context), dim=1))

        return y_pred

    def _init_states(self, X):
        """
        Initialize all 0 hidden states and cell states for encoder.

        Parameters
        ----------
        X: `numpy.ndarray`
            The input array

        Returns
        -------
            initial_hidden_states
        """

        # hidden state and cell state [num_layers*num_directions, batch_size, hidden_size]
        # https://pytorch.org/docs/master/nn.html?#lstm
        return Variable(X.data.new(1, X.size(0), self.decoder_num_hidden).zero_())

In [ ]:
class DA_rnn(nn.Module):
    """DARNN model."""

    def __init__(self, X, y, T,
                 encoder_num_hidden,
                 decoder_num_hidden,
                 batch_size,
                 learning_rate,
                 epochs,
                 loss_func,
                 train_size,
                 parallel=False):
        """
        DA_RNN model initialization.
        
        Parameters
        ----------
        X: `numpy.ndarray`
            The input matrix containing all the timeseries data
        y: `numpy.ndarray`
            The predicted timeseries for the output
        T: `int`
            The number of timesteps to consider attention upon.
        encoder_num_hidden: `int`
            The dimension of the encoder's hidden state
        decoder_num_hidden:: `int`
            The dimension of the decoder's hidden state
        batch_size: `int`
            The batch size
        learning_rate: `float`
            The learning rate to be used
        epochs: `int`
            The number of epochs to run
        loss_func: `nn.Module`
            The loss function to be used
        train_size: `float`
            The percentage of samples to be used for training
        parallel: `bool`
            If set to `True` the training will be done parallely.
        
        Returns
        ------
        None
        """

        super(DA_rnn, self).__init__()
        self.encoder_num_hidden = encoder_num_hidden
        self.decoder_num_hidden = decoder_num_hidden
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.parallel = parallel
        self.shuffle = False
        self.epochs = epochs
        self.T = T
        self.X = X[:,1:]
        self.x_full = X
        self.y_full = y
        self.y = y
        self.train_size = train_size

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        print("==> Use accelerator: ", self.device)

        # Initialize the Encoder and Decoder
        self.Encoder = Encoder(input_size=X.shape[1]-1,
                               encoder_num_hidden=encoder_num_hidden,
                               T=T).to(self.device)
        self.Decoder = Decoder(encoder_num_hidden=encoder_num_hidden,
                               decoder_num_hidden=decoder_num_hidden,
                               T=T).to(self.device)

        # Loss function
        self.criterion = loss_func()

        if self.parallel:
            self.encoder = nn.DataParallel(self.encoder)
            self.decoder = nn.DataParallel(self.decoder)

        # Declare the opimizers
        self.encoder_optimizer = optim.Adam(params=filter(lambda p: p.requires_grad,
                                                          self.Encoder.parameters()),
                                            lr=self.learning_rate)
        self.decoder_optimizer = optim.Adam(params=filter(lambda p: p.requires_grad,
                                                          self.Decoder.parameters()),
                                            lr=self.learning_rate)

        # Training set
        #self.train_timesteps = int(self.X.shape[0] * self.train_size)
        self.train_timesteps = int(251 * self.train_size)  
        # self.y = self.y - np.mean(self.y[:self.train_timesteps])
        self.input_size = X.shape[1]-1


    def train(self, train_summary_writer):
        """
        The training process.

        Parameters
        ----------
        train_summary_writer
            Tensorboard summary writer
        """

        iter_per_epoch = int(np.ceil(self.train_timesteps * 1. / self.batch_size))
        self.iter_losses = np.zeros(self.epochs * iter_per_epoch)
        self.epoch_losses = np.zeros(self.epochs)

        n_iter = 0
        num_geouids = np.unique(self.x_full[:,0], axis = 0).shape[0]

        # for i in range(num_geouids):
        #   self.X = self.x_full[ i*251 : (i+1)*251 ,1:]
        #   self.y = self.y_full[i*251: (i+1)*251]
        #   for epoch in range(self.epochs):
        for epoch in range(self.epochs):
          for i in range(num_geouids):
            self.X = self.x_full[ i*251 : (i+1)*251 ,1:]
            self.y = self.y_full[i*251: (i+1)*251]
            if self.shuffle:
                ref_idx = np.random.permutation(self.train_timesteps - self.T)
            else:
                ref_idx = np.array(range(self.train_timesteps - self.T))

            idx = 0

            while (idx < self.train_timesteps):
                # get the indices of X_train
                indices = ref_idx[idx:(idx + self.batch_size)]
                # x = np.zeros((self.T - 1, len(indices), self.input_size))
                x = np.zeros((len(indices), self.T - 1, self.input_size))
                y_prev = np.zeros((len(indices), self.T - 1))
                y_gt = self.y[indices + self.T]

                # format x into 3D tensor
                for bs in range(len(indices)):
                    x[bs, :, :] = self.X[indices[bs]:(indices[bs] + self.T - 1), :]
                    y_prev[bs, :] = self.y[indices[bs]: (indices[bs] + self.T - 1)]

                loss = self.train_forward(x, y_prev, y_gt)
                self.iter_losses[int(epoch * iter_per_epoch + idx / self.batch_size)] = loss

                idx += self.batch_size
                n_iter += 1

                if n_iter % 4000 == 0 and n_iter != 0:
                    for param_group in self.encoder_optimizer.param_groups:
                        param_group['lr'] = param_group['lr'] * 0.9
                    for param_group in self.decoder_optimizer.param_groups:
                        param_group['lr'] = param_group['lr'] * 0.9

                self.epoch_losses[epoch] = np.mean(self.iter_losses[range(
                    epoch * iter_per_epoch, (epoch + 1) * iter_per_epoch)])
            
            y_train_pred = self.test(on_train=True)
            y_test_pred = self.test(on_train=False)[1:]

            y_train_true = self.y[self.T : (len(y_train_pred) + self.T)]
            y_test_true = self.y[self.T + len(y_train_pred) : (len(self.y) + 1)]

            err_train = np.abs(y_train_true - y_train_pred)
            mae_train = np.mean(err_train)
            rmse_train = np.sqrt(np.mean(err_train ** 2))

            err_test = np.abs(y_test_true - y_test_pred)
            mae_test = np.mean(err_test)
            rmse_test = np.sqrt(np.mean(err_test ** 2))
            
            y_pred = np.concatenate((y_train_pred, y_test_pred))

            err_total = np.abs(self.y[self.T:]-y_pred)
            mae_total = np.mean(err_total)
            rmse_total = np.sqrt(np.mean(err_total ** 2))

            with train_summary_writer.as_default():
                # summary.scalar('Loss', loss, step=epoch)
                summary.scalar('Epoch Average Loss', self.epoch_losses[epoch], step=epoch)
                summary.scalar('MAE_total', mae_total, step=epoch)
                summary.scalar('RMSE_total', rmse_total, step=epoch)
                summary.scalar('MAE_train', mae_train, step=epoch)
                summary.scalar('RMSE_train', rmse_train, step=epoch)
                summary.scalar('MAE_test', mae_test, step=epoch)
                summary.scalar('RMSE_test', rmse_test, step=epoch)
                

          print("Epochs: ", epoch, " Iterations: ", n_iter,
                  " Epoch Loss: ", self.epoch_losses[epoch], " Train MAE", mae_train, 
                  " Train RMSE", rmse_train, " Test MAE: ", mae_test, " Test RMSE", rmse_test)

        y_train_pred = self.test(on_train=True)
        y_test_pred = self.test(on_train=False)

        y_pred = np.concatenate((y_train_pred, y_test_pred))
        plt.ioff()
        plt.figure()
        plt.plot(range(1, 1 + len(self.y)), self.y, label="True")
        plt.plot(range(self.T, len(y_train_pred) + self.T),
                    y_train_pred, label='Predicted - Train')
        plt.plot(range(self.T + len(y_train_pred), len(self.y) + 1),
                    y_test_pred, label='Predicted - Test')
        plt.legend(loc='upper left')
        plt.savefig("test.png")
        img = plt.imread("test.png")
        os.remove("test.png")

            # # Save files in last iterations
            # if epoch == self.epochs - 1:
            #     np.savetxt('../loss.txt', np.array(self.epoch_losses), delimiter=',')
            #     np.savetxt('../y_pred.txt',
            #                np.array(self.y_pred), delimiter=',')
            #     np.savetxt('../y_true.txt',
            #                np.array(self.y_true), delimiter=',')
        
        with torch.no_grad():
            # input_attn = self.Encoder.encoder_attn.weight.cpu().numpy()
            # temp_attn = self.Decoder.attn_layer[2].weight.cpu().numpy()
            # print(input_attn.shape)
            # print(temp_attn.shape)
            with train_summary_writer.as_default():
                # summary.histogram('Input Attention Weights', input_attn, step=1)
                # summary.histogram('Temporal Attention Weights', temp_attn, step=1)
                summary.image('Final Prediction Plot', np.expand_dims(img, 0), step=1)


    def train_forward(self, X, y_prev, y_gt):
        """
        Forward pass through the encoder decoder network.

        Parameters
        ----------
        X: `numpy.ndarray`
            The input array
        y_prev: `numpy.ndarray`
            The previous predicted value
        y_gt: `numpy.ndarray`
            Ground truth label
        
        Returns
        -------
            The loss incurred at the current step
        """

        # zero gradients
        self.encoder_optimizer.zero_grad()
        self.decoder_optimizer.zero_grad()

        input_weighted, input_encoded = self.Encoder(
            Variable(torch.from_numpy(X).type(torch.FloatTensor).to(self.device)))
        y_pred = self.Decoder(input_encoded, Variable(
            torch.from_numpy(y_prev).type(torch.FloatTensor).to(self.device)))

        y_true = Variable(torch.from_numpy(
            y_gt).type(torch.FloatTensor).to(self.device))

        y_true = y_true.view(-1, 1)
        loss = self.criterion(y_pred, y_true)
        loss.backward()

        self.encoder_optimizer.step()
        self.decoder_optimizer.step()

        return loss.item()


    def test(self, on_train=False, inner_test=True, X_subset = None, y_subset = None):
        """
        The test function

        Parameters
        ----------
        on_train: `bool`
            Whether to test on the training data or not
        
        Returns
        -------
            The predicted value
        """

        if on_train:
            y_pred = np.zeros(self.train_timesteps - self.T + 1)
        elif inner_test:
            y_pred = np.zeros(self.X.shape[0] - self.train_timesteps)
        else:
          # redefine self.x and self.y. Is that it?
          self.X = X_subset
          self.y = y_subset
          y_pred = np.zeros(self.X.shape[0] - self.train_timesteps)

        i = 0
        while i < len(y_pred):
            batch_idx = np.array(range(len(y_pred)))[i: (i + self.batch_size)]
            X = np.zeros((len(batch_idx), self.T - 1, self.X.shape[1]))
            y_history = np.zeros((len(batch_idx), self.T - 1))

            for j in range(len(batch_idx)):
                if on_train:
                    X[j, :, :] = self.X[range(
                        batch_idx[j], batch_idx[j] + self.T - 1), :]
                    y_history[j, :] = self.y[range(
                        batch_idx[j], batch_idx[j] + self.T - 1)]
                else:
                    X[j, :, :] = self.X[range(
                        batch_idx[j] + self.train_timesteps - self.T, batch_idx[j] + self.train_timesteps - 1), :]
                    y_history[j, :] = self.y[range(
                        batch_idx[j] + self.train_timesteps - self.T, batch_idx[j] + self.train_timesteps - 1)]

            y_history = Variable(torch.from_numpy(
                y_history).type(torch.FloatTensor).to(self.device))
            _, input_encoded = self.Encoder(
                Variable(torch.from_numpy(X).type(torch.FloatTensor).to(self.device)))
            y_pred[i:(i + self.batch_size)] = self.Decoder(input_encoded,
                                                           y_history).cpu().data.numpy()[:, 0]
            i += self.batch_size

        return y_pred

In [ ]:
def main(debug=False):
    """
    This is the main function which runs the entire pipeline of loading the data, defining the number of
    training runs on the network, along with storing the model performance while training and testing for
    easier access through tensorboard.

    Parameters
    ----------
    debug: `bool`
        If true the training is done in debug mode for a very small number of epochs.
    """

    dataroot = 'full_data.csv'

    # Create the SummaryWriter object
    train_summary_writer = SummaryWriter(log_dir='logs/train')
    
    # Define the list of hyperparameteres to run the training loops on and then the best possible
    # set of hyperparameters can be chosen
    parameters = dict(  batchsize = [32],
                        nhidden_encoder = [32],
                        nhidden_decoder = [32],
                        ntimestep = [12],
                        lr = [0.001],
                        epochs = [5],
                        # loss_func = [CustomLoss2, CustomLoss3],
                        loss_func = [CustomLoss3],
                        diff = [False]
                    )

    param_values = [v for v in parameters.values()]
    print("Number of runs: ", len(list(product(*param_values))))

    run_stats = []
    runs = 0

    for batchsize, nhidden_encoder, nhidden_decoder, ntimestep, lr, epochs, loss_func, diff in product(*param_values):
#batchsize, nhidden_encoder, nhidden_decoder, ntimestep, lr, epochs, loss_func, diff = [32,32,32,12,0.001,1,CustomLoss1,False]


        # Read dataset
        print("==> Load dataset ...")
        province_data = read_data(dataroot, debug, diff)
        # province_data = read_data(dataroot, True, diff)
        train_size = 0.75
        
        runs += 1
        if debug:
            epochs = 2
        
        # Initialize model
        print("==> Initialize DA-RNN model ...")

        #for prov in province_data:
        for prov in province_data:
          model = DA_rnn(
              prov[0],
              prov[1],
              ntimestep,
              nhidden_encoder,
              nhidden_decoder,
              batchsize,
              lr,
              epochs,
              loss_func,
              train_size,
          )

          # Name the run based on hyperparamter values
          run_name = f'batch_size={batchsize} lr={lr} epochs = {epochs} nhidden_encoder = {nhidden_encoder} nhidden_decoder = {nhidden_decoder}'
          run_name += f' ntimestep = {ntimestep} loss_func = {loss_func} diff = {diff}'

          if debug:
              run_name += ' Test'

          # Store the logs
          train_log_dir = 'logs/train/' + run_name
          train_summary_writer = summary.create_file_writer(train_log_dir)

          # Train
          print("==> Start training ...", runs)
          print(run_name)
          model.train(train_summary_writer)

          print("==> Testing ...")

          num_geouids = np.unique(prov[0][:,0], axis = 0).shape[0]
          err_list = []
          mae_list = []
          rmse_list = []

          for i in range(num_geouids):
            sub_X = prov[0][ i*251 : (i+1)*251 ,1:]
            sub_y = prov[1][i*251: (i+1)*251]

            # Prediction
            y_pred = model.test(inner_test=False, X_subset = sub_X, y_subset = sub_y)

            # Test statisitcs

            # Modify to calculate errors for each GeoUID and then average over all of them
            #y_true = prov[1][int(X.shape[0] * train_size):]
            y_true = sub_y[int(sub_X.shape[0] * train_size):]

            err = np.abs(y_true - y_pred)
            mae = np.mean(err)
            rmse = np.sqrt(np.mean(err ** 2))
            
            # Write training loss to TensorBoard
            with train_summary_writer.as_default():
              summary.scalar('Training Loss', np.mean(err), step=i)
              summary.scalar('Training MAE', np.mean(mae), step=i)
              summary.scalar('Training RMSE', np.mean(rmse), step=i)
              
            # Append Errors
            err_list.append(err)
            mae_list.append(mae)
            rmse_list.append(rmse)



          
          avg_err = sum(err_list)/len(err_list)
          avg_mae = sum(mae_list)/len(mae_list)
          avg_rmse = sum(rmse_list)/len(rmse_list)
          #print("Average mean absolute error: ", mae, "Average Root mean Square error: ", rmse)
          print("Average mean absolute error: ", avg_mae, "Average Root mean Square error: ", avg_rmse)
          run_stats.append((avg_rmse, avg_mae, run_name))

          # fig1 = plt.figure()
          # plt.semilogy(range(len(model.iter_losses)), model.iter_losses)
          # plt.savefig("1.png")
          # plt.close(fig1)

          # fig2 = plt.figure()
          # plt.semilogy(range(len(model.epoch_losses)), model.epoch_losses)
          # plt.savefig("2.png")
          # plt.close(fig2)

          # fig3 = plt.figure()
          # plt.plot(y_pred, label='Predicted')
          # plt.plot(model.y[model.train_timesteps:], label="True")
          # plt.legend(loc='upper left')
          # plt.savefig("3.png")
          # plt.close(fig3)
          # print('Finished Training')


    # Display the best results
    run_stats.sort()
    print("Best runs: ")
    for run in run_stats:
        print("Name:", run[2], "RMSE ", run[0], "MAE: ", run[1])



In [ ]:
main()

In [ ]:
!pip install tensorboardX

In [ ]:
import tensorboardX
%load_ext tensorboard
%tensorboard --logdir logs
